In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 3"

# Code

In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import json
import pandas as pd
from tqdm import tqdm
import logging
import time
import code
import code_ablation
from collections import defaultdict
from datasets import load_dataset
from datasets import Dataset
model = "meta-llama/Llama-2-7b-chat-hf"
# model = "meta-llama/Llama-2-13b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


/home/yanghn/env_py/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.55s/it]


In [3]:
def ask(question:str) -> str:
    sequences = pipeline(
        question,
        do_sample=False,
        # top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=400,
    )
    for seq in sequences:
        return seq['generated_text']

In [4]:
def list_2_str_num(facts:list) -> str:
    res = ""
    for index, fact in enumerate(facts):
        res += f"{index+1}. {fact}\n"
    return res
def list_2_str(facts:list) -> str:
    res = ""
    for index, fact in enumerate(facts):
        res += f"{fact}\n"
    return res


In [5]:
"123".split('q')

['123']

In [6]:
def result_extractor_inference(prompt, result):
    result = result.strip()
    if prompt in result:
        result = result.replace(prompt, "")
    if '</Answer>' in result:
        result = result.split('</Answer>')[0].strip()
    result_lines = result.split('\n')
    answer_label_line = result_lines[0]
    if len(result_lines) > 1:
        answer_conclusion_line = result_lines[1]
    else:
        answer_conclusion_line = ''

    if 'yes' in answer_label_line.lower() and 'no' not in answer_label_line.lower():
        answer_label = 'yes'
    elif 'yes' not in answer_label_line.lower() and 'no' in answer_label_line.lower():
        answer_label = 'no'
    else:
        answer_label = 'error'

    answer_conclusion_after_produce = answer_conclusion_line.split('Produce:')
    if len(answer_conclusion_after_produce) > 1:
        answer_conclusion = answer_conclusion_after_produce[1].strip()
    else:
        answer_conclusion = answer_conclusion_after_produce[0]
    
    return answer_label, [answer_conclusion], answer_conclusion_line
    

In [7]:
from datasets import load_dataset
from datasets import Dataset


def evaluate_dev(prompt_answer_dict_list:list):

    p_acc   = 0
    p_acc_T = 0
    p_acc_F = 0
    c_acc   = 0
    c_acc_T = 0
    c_acc_F = 0
    b_acc   = 0
    b_acc_T = 0
    b_acc_F = 0
    n_err   = 0
    n       = 0
    num_NOTHING = 0
    num_NOTHING_T = 0
    num_NOTHING_F = 0

    print(f"[nr] [pred] [conc]; [gold] [conc] [full]")
    for prompt_answer in prompt_answer_dict_list:
            prompt      = prompt_answer['prompt']
            gold_answer = prompt_answer['answer']
            conclusion = prompt_answer['conclusion']
            correct_pred = False
            correct_conc = False
            error   = False

            answer  = ask(prompt)
            answer_label, answer_conclusion, answer_full = result_extractor_inference(prompt, answer)
            if answer_label == 'error':
                error = True
            elif answer_label == gold_answer:
                correct_pred = True
            if answer_conclusion == conclusion:
                correct_conc = True

            # print(f"[{n}] [{'correct' if correct_pred else 'error' if error else 'wrong'}] [{'correct' if correct_conc else 'wrong'}] [{gold_answer}/{answer_label}] [{conclusion}/{answer_conclusion}] {answer_full}")
            # compare with True Answer
            n += 1
            if answer_conclusion == 'NOTHING':
                num_NOTHING += 1
                if gold_answer == 'yes':
                    num_NOTHING_T += 1
                elif gold_answer == 'no':
                    num_NOTHING_F += 1

            if error:
                 n_err += 1
            if correct_pred:
                p_acc += 1
                if gold_answer == 'yes':
                    p_acc_T += 1
                elif gold_answer == 'no':
                    p_acc_F += 1
            
            if correct_conc:
                c_acc += 1
                if gold_answer == 'yes':
                    c_acc_T += 1
                elif gold_answer == 'False':
                    c_acc_F += 1
            
            if correct_pred and correct_conc:
                b_acc += 1
                if gold_answer == 'yes':
                    b_acc_T += 1
                elif gold_answer == 'no':
                    b_acc_F += 1

            

    print(f"""Pred_Acc*:\t{p_acc}/100, 
Pred_TAcc*:\t{p_acc_T}/50, {p_acc_T*2}%
Pred_FAcc*:\t{p_acc_F}/50, {p_acc_F*2}%

Conc_Acc:\t{c_acc}/100,
Conc_TAcc*:\t{c_acc_T}/50, {c_acc_T*2}%
Conc_FAcc:\t{c_acc_F}/50, {c_acc_F*2}%

Both_Acc:\t{b_acc}/100,
Both_TAcc:\t{b_acc_T}/50, {b_acc_T*2}%
Both_FAcc:\t{b_acc_F}/50, {b_acc_F*2}%

num_NOTHING:\t{num_NOTHING}/100,
num_NOTHING_T:\t{num_NOTHING_T}/50, {num_NOTHING_T*2}%
num_NOTHING_F:\t{num_NOTHING_F}/50, {num_NOTHING_F*2}%

Err:\t{n_err}/100""")



In [16]:
def result_extractor_inference_multi(prompt, result):
    result = result.strip()
    if prompt in result:
        result = result.replace(prompt, "")
    if '</Answer>' in result:
        result = result.split('</Answer>')[0].strip()
    result_lines = result.split('\n')
    result_line = result_lines[0]
    results = result_line.split(';')
    res = []
    for result in results:
        result = result.strip()
        if '. ' in result:  # 1. sentence
            result = result.split('.')[1]
            result = result.strip()
        elif '.' in result: # sentence.
            result = result.split('.')[0]
            result = result.strip()
        res.append(result)
    return "", res, result_line

# Mid

# Inference_single

## Bad Over all

1. instruction satisfication
2. example
3. 

In [8]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no', and answer answer 'NOTHING'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: {facts_str}
Rule: {rules_str},
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no', and answer answer 'NOTHING'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: Bob is good
Rule: If Bob is good then Bob is nice,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: no
Produce: NOTHING
</Answer>

Please answer the following questions:

1. Facts: Sarah is tall
Rule: If Sarah is tall then Sarah is happy.
Answer: [yes or no]
Produce: [your answer here]

2. Facts: John is old
Rule: If John is old then John is grumpy.
Answer: [yes or no]
Produce: [your answer here]

3. Facts: Lisa is thin
Rule: If Lisa is thin then Lisa is healthy.
Answer: [yes or no]
Produce: [your answer here]

4. Facts: Michael is smart
Rule: If Michael is smart then Michael is successful.
Answer: [yes or no]
Produce

In [9]:
from datasets import load_dataset
from datasets import Dataset

In [15]:
Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no', and answer answer 'NOTHING'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: 1. the rabbit is round
2. the rabbit needs the squirrel

Rule: the rabbit is rough,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:
[nr] [pred] [conc]; [gold] [conc] [full]


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[0] [wrong] [wrong] [yes/no] [the rabbit is rough/NOTHING] Produce: NOTHING
[1] [wrong] [wrong] [yes/no] [the cat chases the bear/NOTHING] Produce: NOTHING
[2] [correct] [correct] [yes/yes] [the bear likes the cat/the bear likes the cat] Produce: the bear likes the cat
[3] [correct] [wrong] [yes/yes] [Bob is kind/NOTHING] Produce: NOTHING
[4] [correct] [correct] [yes/yes] [the cat visits the dog/the cat visits the dog] Produce: the cat visits the dog
[5] [wrong] [wrong] [yes/no] [the bear sees the dog/NOTHING] Produce: NOTHING
[6] [wrong] [wrong] [yes/no] [Bob is green/NOTHING] Produce: NOTHING
[7] [wrong] [wrong] [yes/no] [the mouse visits the lion/NOTHING] Produce: NOTHING
[8] [wrong] [wrong] [yes/no] [the dog is round/NOTHING] Produce: NOTHING
[9] [wrong] [wrong] [yes/no] [the tiger eats the squirrel/NOTHING] Produce: NOTHING
[10] [wrong] [wrong] [yes/no] [Bob is blue/NOTHING] Produce: NOTHING
[11] [wrong] [wrong] [yes/no] [the lion is rough/NOTHING] Produce: NOTHING
[12] [wrong] [w

## Bad Conclusion Without NOTHING

In [17]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: {facts_str}
Rule: {rules_str},
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: Bob is good
Rule: If Bob is good then Bob is nice,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: yes
Produce: Bob is nice
</Answer>
Please answer the next question.
Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes' and answer what does it produce. if no, say 'no'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: 1. the rabbit is round
2. the rabbit needs the squirrel

Rule: the rabbit is rough,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:
[nr] [p

## Bad Conclusion

If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce, else answer 'NOTHING'.

In [18]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: {facts_str}
Rule: {rules_str},
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce, else answer 'NOTHING'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: Bob is good
Rule: If Bob is good then Bob is nice,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: yes
Produce: Bob is nice
</Answer>
Please answer the next question.
Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce, else answer 'NOTHING'.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
</Example>
Facts: 1. the rabbit is round
2. the rabbit needs the squirrel

Rule: the rabbit is rough,
Please answer with 

## Negatie Example. bad prediction False

In [19]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, 
Procude: NOTHING
</Example>
Facts: {facts_str}
Rule: {rules_str},
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, 
Procude: NOTHING
</Example>
Facts: Bob is good
Rule: If Bob is good then Bob is nice,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: yes
Produce: Bob is nice
</Answer>

Please answer the following questions:

1. Is Bob good?
Answer: [yes or no]
Produce: [your answer here]
2. Is Bob nice?
Answer: [yes or no]
Produce: [your answer here]
3. Is Erin round?
Answer: [yes or no]
Produce: [your answer here]
4. Is Erin white?
Answer: [yes or no]
Produce: [your answer here]

Please answer the questions and I will provide you with the next set of facts and rules.
Task: pleas

## instruction if-then, bad prediction False

In [22]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If a "if <condition> then <conclusion>" rule is satisfied, the <conclusion> will be produced.
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, 
Procude: NOTHING
</Example>
Facts: {facts_str}.
Rule: {rules_str}.Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: please answer whether the following rule is satisfied under the provided given Facts?
If a "if <condition> then <conclusion>" rule is satisfied, the <conclusion> will be produced.
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, 
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, 
Procude: NOTHING
</Example>
Facts: Bob is good.
Rule: If Bob is good then Bob is nice.Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: yes
Produce: Bob is nice.
</Answer>

Please answer the following questions:

1. Is Bob good?
Answer: [yes or no]
Produce: [your answer here]
2. Is Erin round?
Answer: [yes or no]
Produce: [your answer here]
3. Is Erin white?
Answer: [yes or no]
Produce: [your answer here]

Please answer the questions and I will provide you with the next set 

## reason - good

In [24]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, the condition can be found in facts.
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, the condition can't be found in facts.
Procude: NOTHING
</Example>
Facts: {facts_str}
Rule: {rules_str},
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer:"""
    return prompt

facts='Bob is good'
rules='If Bob is good then Bob is nice'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)

Dataset.cleanup_cache_files
path_dev = "./CWA_rules_100_TF.jsonl"
dataset = load_dataset('json', data_files=path_dev)

prompt_answer_dict_list = [
    {
        'prompt': ablation_inference_prompt_formulate_nl(
                list_2_str_num(data['conditions']),
                data['conclusion']), 
        'answer':data['answer'],
        'conclusion':data['conclusion'],
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: please answer whether the following rule is satisfied under the provided given Facts?
If yes, say 'yes', if no, say 'no',
If it is satisfied, please answer what does it produce.
<Example>
Facts: Erin is round
Rule: If Erin is round then Erin is white.
Answer: yes, the condition can be found in facts.
Procude: Erin is white
Facts: Erin is kind
Rule: If Erin is round then Erin is white.
Answer: no, the condition can't be found in facts.
Procude: NOTHING
</Example>
Facts: Bob is good
Rule: If Bob is good then Bob is nice,
Please answer with the following format:
Answer: [yes or no]
Produce: [your answer here]
<Answer>
Answer: yes
Produce: Bob is nice
</Answer>

Please answer the following questions:

1. Is Bob round?
Answer: [yes or no]
Produce: [your answer here]
2. Is Bob white?
Answer: [yes or no]
Produce: [your answer here]

Please answer the following questions:

1. Is Erin round?
Answer: [yes or no]
Produce: [your answer here]
2. Is Erin white?
Answer: [yes or no]
Produce: [yo

# Inference Multi

## BAD

In [17]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: Erin is round, Erin is nice
Rule: If Erin is round then Erin is white. If Erin is nice then Erin is cute.
Answer: Erin is white. Erin is cute.
Facts: Bob is kind; Bob is rude; Bob is rough;
Rule: If Bob is kind then Bob is blue; If Bob is rough, then Bob is nice; If Bob is cute then Bob is kind;
Answer: Bob is blue; Bob is nice; 
</Example>
Facts: {facts_str}
Rule: {rules_str} Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer:"""
    return prompt

facts='Bob is good. Bob is cute.'
rules='If Bob is good then Bob is nice. If Bob is cute then Bob is white.'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)


Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: Erin is round, Erin is nice
Rule: If Erin is round then Erin is white. If Erin is nice then Erin is cute.
Answer: Erin is white. Erin is cute.
Facts: Bob is kind; Bob is rude; Bob is rough;
Rule: If Bob is kind then Bob is blue; If Bob is rough, then Bob is nice; If Bob is cute then Bob is kind;
Answer: Bob is blue; Bob is nice; 
</Example>
Facts: Bob is good. Bob is cute.
Rule: If Bob is good then Bob is nice. If Bob is cute then Bob is white. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer: Bob is white.


## Good

In [18]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: {facts_str}.
Rule: {rules_str}. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer:"""
    return prompt

facts='1. Bob is good; 2. Bob is cute.'
rules='1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white.'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)


Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: 1. Bob is good; 2. Bob is cute.
Rule: 1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer: Bob is nice; Bob is white.
</Answer>

Please answer the question with the given format.


## Bad

In [19]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: {facts_str}
Rule: {rules_str} 
Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer:"""
    return prompt

facts="""1. Bob is good; 
2. Bob is cute."""
rules="""1. If Bob is good then Bob is nice; 
2. If Bob is cute then Bob is white."""
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: 1. Bob is good; 
2. Bob is cute.
Rule: 1. If Bob is good then Bob is nice; 
2. If Bob is cute then Bob is white. 
Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer: Erin is white.
</Answer>
<Answer>
Answer: Bob is blue.
</Answer>
<Answer>
Answer: Bob is nice.
</Answer>
<Answer>
Answer: Bob is white.
</Answer>
<Answer>
Answer: Erin is cute.
</Answer>
<Answer>
Answer: Bob is nice.
</Answer>
<Answer>
Answer: Bob is white.
</Answer>
<Answer>
Answer: Erin is cute.
</Answer>
<Answer>

# No facts (Bad)

In [10]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: {facts_str}.
Rule: {rules_str}. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer:"""
    return prompt

facts=''
rules='1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white.'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)


Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
</Example>
Facts: .
Rule: 1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white.. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer: Bob is nice; Bob is white.
</Answer>

Answer: Bob is nice; Bob is white.


# No facts with NOTHING Example (Good)

In [9]:
def ablation_inference_prompt_formulate_nl(facts_str:str, rules_str:str):
    prompt = f"""Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
Facts: 1. Bob is kind; 2. Bob is rude;
Rule: 1. If Bob is pretty then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: NOTHING
</Example>
Facts: {facts_str}.
Rule: {rules_str}. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer:"""
    return prompt

facts=''
rules='1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white.'
prompt = ablation_inference_prompt_formulate_nl(facts, rules)
answer = ask(prompt)
print(answer)


Task: please answer what can be produce from the rules with given facts?
<Example>
Facts: 1. Erin is round; 2. Erin is nice;
Rule: 1. If Erin is round then Erin is white; 2. If Erin is nice then Erin is cute;
Answer: 1. Erin is white; 2. Erin is cute;
Facts: 1. Bob is kind; 2. Bob is rude; 3. Bob is rough;
Rule: 1. If Bob is kind then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: 1. Bob is blue; 2. Bob is nice; 
Facts: 1. Bob is kind; 2. Bob is rude;
Rule: 1. If Bob is pretty then Bob is blue; 2. If Bob is rough, then Bob is nice; 3. If Bob is cute then Bob is kind;
Answer: NOTHING
</Example>
Facts: .
Rule: 1. If Bob is good then Bob is nice; 2. If Bob is cute then Bob is white.. Please answer with the following format:
Answer: [your answer here].
<Answer>
Answer: 1. Bob is nice; 2. Bob is white.


In [17]:
result_extractor_inference_multi(prompt, answer)

('', ['Bob is nice', 'Bob is white'], 'Bob is nice; Bob is white.')